In [38]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [39]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

/bin/sh: pip: command not found
/bin/sh: pip: command not found


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 
              'leagues', 'news', 'streamingplatforms', 'techgiants']

subCategories = [['output_AmericanAir', 'output_JetBlue', 'output_qatarairways', 'output_SingaporeAir'], 
                 ['output_TheRock', 'output_ArianaGrande', 'output_chrissyteigen', 'output_jimmyfallon'], 
                 ['output_Stanford', 'output_Harvard', 'output_MIT', 'output_UCBerkeley'], 
                 ['output_McDonalds', 'output_BurgerKing', 'output_Starbucks', 'output_wendys'], 
                 ['output_IPL', 'output_MLB', 'output_MLS', 'output_NBA'], 
                 ['output_BBCWorld', 'output_CNN', 'output_washingtonpost', 'output_nytimes'], 
                 ['output_Netflix', 'output_Spotify', 'output_PrimeVideo', 'output_HBO'], 
                 ['output_amazon', 'output_facebook', 'output_Tesla', 'output_Microsoft']]

In [41]:
sizeOfSets = 400

categCount = 0
for i in range(len(categories)):
    for j in subCategories[i]:
        categCount += 1
        
categCount *= sizeOfSets

val = [None] * categCount # Keep value (sizeOfSets * no. of categories)
sizeOfTweetContent = 75

print(len(val)) # No. Of Posts

12800


In [42]:
def inputDataInDataframe():
    overCount= 0
    for i in range(len(categories)):
        for j in subCategories[i]:
            direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+categories[i]+'/'+j+'.csv'
            with open(direc, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                count = 0
                for line in csv_reader:
                    if len(line[6]) > sizeOfTweetContent:
                        val[overCount] = line
                        count += 1
                        overCount += 1
                        if count >= sizeOfSets :
                            break

In [43]:
inputDataInDataframe()

In [44]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink']) 

In [45]:
# Once data added in dataframe, Shuffle it
df = df.sample(frac=1).reset_index(drop=True)
# Add a unique id (rec_id) for all tweets
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [46]:
df = df[['rec_id', 'username', 'text', 'date', 'retweets']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12800 entries, 0 to 12799
Data columns (total 5 columns):
rec_id      12800 non-null int64
username    12800 non-null object
text        12800 non-null object
date        12800 non-null object
retweets    12800 non-null object
dtypes: int64(1), object(4)
memory usage: 600.0+ KB


In [47]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

12800

In [48]:
# Extract TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(12800, 26984)

In [49]:
# Compute Pairwise Document Similarity

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12790,12791,12792,12793,12794,12795,12796,12797,12798,12799
0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000
1,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000
2,0.0,0.0,1.0,0.0,0.0,0.0,0.045316,0.0,0.0,0.063435,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054328,0.0000
3,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000
4,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.042222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0105


In [50]:
# Get list of post IDs

posts_list = df['rec_id'].values
posts_list, posts_list.shape

(array([    0,     1,     2, ..., 12797, 12798, 12799]), (12800,))

In [51]:
posts_idx = np.where(posts_list == 3997)[0][0]
posts_idx

3997

In [52]:
posts_similarities = doc_sim_df.iloc[posts_idx].values
posts_similarities

array([0., 0., 0., ..., 0., 0., 0.])

In [53]:
# Get top 5 similar post IDs

similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
similar_posts_idxs

array([ 8279, 12389, 12494,  4662,  1550])

In [87]:
similar_posts = posts_list[similar_posts_idxs]
similar_posts

array([ 8279, 12389, 12494,  4662,  1550])

In [88]:
def relevant_posts(postId, returnSize, posts = posts_list, doc_sims = doc_sim_df):
    # find post id
    posts_idx = np.where(posts == postId)[0][0]
    # get posts similarities
    posts_similarities = doc_sims.iloc[posts_idx].values
    # get top 5 similar post IDs
    similar_posts_idxs = np.argsort(-posts_similarities)[1:returnSize+1]
    # get top 5 posts
    similar_posts = posts[similar_posts_idxs]
    # return the top 5 posts
    return similar_posts

In [140]:
post_id = 9000
returnValSize = 5

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, returnSize = returnValSize, posts = posts_list, doc_sims = doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

-----Input Post-----
The JAPAN Trip Navigator app is helping travelers get the most out of their trips with a friendly AI-powered chatbot named Miko. Say konnichiwa: http://msft.social/leFwT6

-----Relevant Posts-----
=> @ShaffranNick We're happy to look into this for you. Smooth honeymoon trips are important, we get that.
=> Cross-country trips are possible with the Supercharger network and it takes around 30 minutes to charge. You can plan your trip here: https://www.tesla.com/trips
=> Enroute to #Langkawi with a special menu for our travelers. #LangkawiTogether
=> Authorities warn of USB outlet-charging scam that steals travelers’ private information
=> I don’t care what u say, this me!!!!! https://www.instagram.com/p/B32aYJgpGDI/?igshid=13z6l39ns32oa


In [143]:
# Sort Posts

post_id = 9000
returnValSize = 15

postsList = []

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, returnSize = returnValSize, posts = posts_list, doc_sims = doc_sim_df):
    postsList.append((df.loc[df['rec_id'] == i]).values[0])

-----Input Post-----
The JAPAN Trip Navigator app is helping travelers get the most out of their trips with a friendly AI-powered chatbot named Miko. Say konnichiwa: http://msft.social/leFwT6

-----Relevant Posts-----


In [144]:
def sortDate(val): 
    return val[3]

def sortRetweet(val): 
    return val[4]

postsList.sort(key=sortDate, reverse=True)

recentList = postsList[0:5]

for i in recentList:
    print("=>", i[2])

=> @ShaffranNick We're happy to look into this for you. Smooth honeymoon trips are important, we get that.
=> Millions of Thanksgiving travelers in the US will get walloped by several storms this week. Here's what we know about the storms:
=> Authorities warn of USB outlet-charging scam that steals travelers’ private information
=> I will create some budget friendly recipes! maybe 20 things to make at home under 10 bucks (multiple servings) - I'm down!
=> Look TruGreen created a chatbot without running code or re-creating their infrastructure. They did it with two new Power Platform capabilities. #MSIgnite
